In [1]:
%%writefile ratings.py

from mrjob.job import MRJob
import os
from functools import reduce
from mrjob.protocol import RawValueProtocol

class BookRatings(MRJob):
    OUTPUT_PROTOCOL = RawValueProtocol
    userdata={}
    
    def mapper(self, _, line):
        file_name = os.getenv('mapreduce_map_input_file').split("//")[1]
        
        if file_name == "BX-Book-Ratings.csv":
            line = line.split(";")
            userid = line[0].strip('\"')
            try : rating = int(line[2].strip('\"'))
            except ValueError: pass 
            else :
                if userid not in self.userdata: self.userdata[userid] = [rating]
                else: self.userdata[userid].append(rating)
            
        if file_name == "BX-Users.csv":
            line.strip('\"')
            line = line.split(";")
            data = line[1].split(",")
            if len(data) >= 1 and line[0] not in ["User-ID"]:
                country = data[-1].strip('\"')
                user = line[0].strip('\"')
                if user in self.userdata:
                    ratings = self.userdata[user]
                    yield country, ratings
         
    def reducer(self, key, counts_iter):
        country_ratings = reduce(lambda x,y :x+y ,counts_iter)
        totalsum = sum(country_ratings)
        totalcount = len(country_ratings)
        avg = totalsum/totalcount
        
        yield key, str(key)[1:]+";"+str(totalcount)+";"+ str(avg)
    
if __name__ == '__main__':
    BookRatings.run()

Overwriting ratings.py


In [2]:
%run ratings.py BX-Book-Ratings.csv BX-Users.csv
#takes about 2 mins, not really optimal. Also lots of garbage, thats big data for you..

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\Petri\AppData\Local\Temp\ratings.Petri.20191128.171607.635741
Running step 1 of 1...
job output is in C:\Users\Petri\AppData\Local\Temp\ratings.Petri.20191128.171607.635741\output
Streaming final output from C:\Users\Petri\AppData\Local\Temp\ratings.Petri.20191128.171607.635741\output...


&#20013;1;9.0
&#22235;1;10.0
&#304;1;0.0
&#322;2;9.0
.;1;0.0
\"n/a\;10;5.3
afghanistan;5;4.2
alachua;3;1.3333333333333333
albania;10;2.6
alderney;6;4.333333333333333
algeria;3;5.333333333333333
america;8;5.0
andorra;51;3.980392156862745
angola;1;8.0
antarctica;60;7.066666666666666
antigua and barbuda;2;9.0
argentina;414;5.345410628019324
aroostook;1;7.0
aruba;129;3.891472868217054
australia;26473;2.775847089487402
austria;4717;2.632817468730125
autralia;1;7.0
az;1;8.0
azerbaijan;6;4.833333333333333
bahamas;32;4.21875
bahrain;13;1.7692307692307692
bangladesh;3;6.333333333333333
barbados;13;6.615384615384615
basilicata;1;10.0
basque country;2;0.0
belgium;805;4.304347826086956
belize;14;4.285714285714286
benin;7;2.4285714285714284
berguedà;3;2.0
bermuda;34;3.9705882352941178
bolivia;6;4.166666666666667
bosnia and herzegovina;2;5.0
brasil;2;4.5
brazil;1503;4.189620758483034
brunei;6;5.833333333333333
bulgaria;207;2.676328502415459
burkina faso;1;0.0
burlington;1;0.0
burma;157;3.49044585987

Removing temp directory C:\Users\Petri\AppData\Local\Temp\ratings.Petri.20191128.171607.635741...
